In [1]:
#librería de manipulación de dataframes
import pandas as pd
#Funciones matemáticas, necesitaremos sólo importar la función sqrt
from math import sqrt





In [37]:
#Guardando la información de la película dentro de un dataframe de panda
movies_df = pd.read_csv('dfdef.csv')
#Guardando información del usuario dentro de un dataframe de panda
ratings_df = pd.read_csv('user.csv')

In [39]:
movies_df.head()

,id,rating_x,type,release_year,rating_y,duration_int,duration_type,listed_in
0,as1,3.467131,movie,2014,G,113.0,min,"comedy, drama"
1,as10,3.439571,movie,1994,G,92.0,min,drama
2,as100,3.609302,movie,1993,pg-13,109.0,min,"comedy, science fiction"
3,as1000,3.556701,movie,2014,16+,90.0,min,"arts, entertainment, and culture, comedy, docu..."
4,as1001,3.585288,movie,2016,g,84.0,min,documentary


In [40]:

ratings_df = ratings_df.drop('Unnamed: 0', 1)

C:\Users\Martin Rodrigo\AppData\Local\Temp\ipykernel_12304\3391248902.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df = ratings_df.drop('Unnamed: 0', 1)


Filtrado Colaborativo en python
Ahora, comenzaremos el trabajo con los sistemas recomendadores.

La primer técnica que vas a ver se llama Filtrado Colaborativo, también conocido como Filtrado de Usuario a Usuario. Como lo indica su nombre alternativo, esta técnica utiliza otros usuarios para recomendar items al ingreso de datos. Se intenta encontrar usuarios que tengan preferencias y opiniones parecidas para entonces recomendar items que se hayan parecido al ingreso anterior. Existen varios métodos para encontrar usuarios parecidos (Incluso los que utilizan Machine Learning), y el método que vamos a utilizar estará basado en la Función de Correlación de Pearson.
El proceso para crear un sistema de recomendación Basado en el Usuario es el siguiente:

Elegir un usuario con las películas que el usuario ha mirado
Basado en su indice de selección de películas, encuentra a los primeros X vecinos
Obtener el registro de la película que miró el usuario para cada vecino.
Calcular un puntaje de similitud utilizando alguna fórmula
Recomendar los ítems con los puntajes más altos

El usuario lo puedo colocar con una funcion que tome las peiculas vistas y puntuadas por un usuario en particular.


In [7]:
def get_user(usuario_ingresado):
    
    userp = ratings_df[ ratings_df['userId'] == usuario_ingresado]
    return userp

In [8]:
userp= get_user(3)


In [9]:
userp

,userId,rating,movieId
49,3,3.0,ns4695
50,3,2.0,as1246
51,3,4.0,hs653
52,3,4.0,ns8641
53,3,3.0,hs1531
54,3,3.0,as8113
55,3,3.0,ns4429
56,3,3.0,ns8253
57,3,3.0,as6156
58,3,3.0,as9380


In [10]:
#Filtrando los usuarios que han visto las películas y guardándolas
inputMovies=userp
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
#RESOLVER# userSubset.userId.remove(userp.userId)
userSubset.info()
#aca debo exptual el valor userId del usuario ingresado


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4806 entries, 49 to 11023623
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   4806 non-null   int64  
 1   rating   4806 non-null   float64
 2   movieId  4806 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 150.2+ KB


In [11]:
#Groupby crea varios dataframes donde todos tienen el mismo valor para la columna especificada como parámetro
userSubsetGroup = userSubset.groupby(['userId'])

In [12]:
#Ordenamiento de forma tal de que los usuarios con más películas en común tengan prioridad
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

C:\Users\Martin Rodrigo\AppData\Local\Temp\ipykernel_12304\3140675019.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)


In [13]:
#SE CORROBORA EL PROCESO ANTERIOR
userSubsetGroup[0:3]


[(45811,
           userId  rating movieId
  4440837   45811     4.5  ns4429
  4442585   45811     4.0  as6156
  4443152   45811     4.0  as6156
  4445428   45811     3.0  ns8253
  4446145   45811     5.0  as6156
  4447298   45811     4.0  as9380
  4449443   45811     2.5  ns8253
  4450139   45811     3.0   hs653
  4450551   45811     2.5  ns4429
  4452105   45811     3.5   hs653
  4452470   45811     3.0  as1246
  4454455   45811     2.5  ns8253
  4455065   45811     3.0  ns4695
  4457677   45811     2.0  ns4695),
 (3,
      userId  rating movieId
  49       3     3.0  ns4695
  50       3     2.0  as1246
  51       3     4.0   hs653
  52       3     4.0  ns8641
  53       3     3.0  hs1531
  54       3     3.0  as8113
  55       3     3.0  ns4429
  56       3     3.0  ns8253
  57       3     3.0  as6156
  58       3     3.0  as9380),
 (4387,
          userId  rating movieId
  421118    4387     3.0  ns4429
  421421    4387     3.0  as1246
  421614    4387     3.5  as8113
  422631    4

## Similitud entre usuarios y usuarios ingresantes

Luego, compararemos a todos los usuarios (casi todos) con nuestro usuario indicado y encontraremos el que más se parece.
Encontraremos cómo cada usuario similar se relacionan entre si a través del Coeficiente de Correlación de Pearson. Se utiliza para medir la fuerza de una asociación lineal entre dos variables. La fórmula para encontrar este coeficiente entre los conjuntos X e Y con los valores de N se puede ver en la fórmula:
r=\frac{\sum_{i=1}^{n}(x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^{n}(x_i-\bar{x})^2}\sqrt{\sum_{i=1}^{n}(y_i-\bar{y})^2}}
¿Por qué la Correlación Pearson?
La correlación Pearson no varía con la escala, ejemplo: si se multiplican todos los elementos por una constante distinta a cero o si se agrega cualquier constante a todos los elementos. Por ejemplo, si tienes dos vectoresXeY, entonces,pearson(X, Y)=pearson(X, 2 \cdot Y + 3). Esta es una propiedad muy importante en los sistemas recomendadores por que dos usuarios podrían putnuar dos series de elementos de manera completamente diferente, pero serían usuarios parecidos (ejemplo: con ideas similares) con puntuaciones parecidas en escalas variadas .

Los valores brindados por la fórumula puede variar der=-1ar=1, donde 1 se correlaciona directamente entre las dos entidades (esto sería una correlación positiva perfecta) y -1 forma una correlación negativa perfecta. En nuestro caso, un 1 se refiere a que dos usuarios tiene gustos parecidos, mientras que -1 es lo opuesto.
Elegiremos un subconjunto de usuarios para hacer las iteraciones. Este limite existe porque no queremos desperdiciar mucho tiempo pasando por cada usuario.

In [14]:
userSubsetGroup = userSubsetGroup[0:1000]

Ahora, calculemos la Correlación Pearson entre la entrada del usuario el grupo, para almacenarlo en el diccionario, donde la clave es el Id del usuario y el valor es el coeficiente

In [15]:
#Guardar la Correlación Pearson en un diccionario, donde la clave es el Id del usuario y el valor es el coeficiente
pearsonCorrelationDict = {}

#Para cada grupo de usuarios en nuestro subconjunto 
for name, group in userSubsetGroup:
    #Comencemos ordenando el usuario actual y el ingresado de forma tal que los valores no se mezclen luego
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtener el N para la fórmula
    nRatings = len(group)
    #Obtener los puntajes de revisión para las películas en común
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #Guardarlas en una variable temporal con formato de lista para facilitar cálculos futuros
    tempRatingList = temp_df['rating'].tolist()
    #Pongamos también las revisiones de grupos de usuarios en una lista
    tempGroupList = group['rating'].tolist()
    #Calculemos la Correlación Pearson entre dos usuarios, x e y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #Si el denominador es diferente a cero, entonces dividir, sino, la correlación es 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [16]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.648371,45811
1,1.000000,3
2,0.514496,4387
3,-0.387943,16481
4,-0.351391,56845


In [17]:
#Ahora obtengamos los 50 primeros usuarios más parecidos a los que se ingresaron.
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
273,1.0,73463
53,1.0,506
187,1.0,46126
189,1.0,46570
59,1.0,5243


Recomendemos películas al usuario de entrada puntuando a los usuarios elegidos para todas las películas

Haremos esto tomando el peso promedio de los ratings de las películas utilizando la Correlación Pearson. Pero para hacer esto, primero necesitamos que los usuarios vean las películas en nuestro pearsonDF a partir del dataframe de puntajes y luego guardar su correlación en una nueva columna llamada _similarityIndex». Estos se logra juntando estas dos tablas de debajo.

In [18]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,rating,movieId
0,1.0,73463,5.0,as5879
1,1.0,73463,3.0,ns8318
2,1.0,73463,3.0,as2864
3,1.0,73463,3.0,ns3438
4,1.0,73463,3.0,as2983


Ahora todo lo que se necesita hacer es multiplicar el puntaje de la película por su peso (El índice de similitud), luego se suman los nuevos puntajes y dividen por la suma de los pesos.
Esto se logra sencillamente multiplicando dos columnas, luego agrupando el dataframe por la columna movieId y luego dividiendo dos columnas:
Aqui se muestra la idea de todos los usuarios similares respecto de las películas candidatas para el usuario ingresado:

In [19]:
#Se multiplica la similitud de los puntajes de los usuarios
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,rating,movieId,weightedRating
0,1.0,73463,5.0,as5879,5.0
1,1.0,73463,3.0,ns8318,3.0
2,1.0,73463,3.0,as2864,3.0
3,1.0,73463,3.0,ns3438,3.0
4,1.0,73463,3.0,as2983,3.0


In [20]:
#Se aplica una suma a los topUsers luego de agruparlos por userId
tempTopUsersRating = topUsersRating.groupby(topUsersRating['movieId']).sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns =['sum_similarityIndex','sum_weightedRating']
#tempTopUsersRating.drop([''], axis=1)
#tempTopUsersRating.drop(['movieId'], axis=1)

#tempTopUsersRating.drop([1],axis=0)
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
as1,3.0,12.0
as10,8.0,29.5
as100,1.0,4.0
as1000,2.0,6.0
as1001,1.0,2.0


In [21]:
#Se crea un dataframe vacío
recommendation_df = pd.DataFrame()
#Ahora se toma el promedio ponderado
recommendation_df['weighted_average_recommendation_score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
#recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted_average_recommendation_score
movieId,
as1,4.0000
as10,3.6875
as100,4.0000
as1000,3.0000
as1001,2.0000


In [22]:
#Luego, ordenémoslo y veamos las primeras 20 películas que el algoritmo recomendó!
recommendation_df = recommendation_df.sort_values(by='weighted_average_recommendation_score', ascending=False)
recommendation_df['movieId'] = recommendation_df.index
recommendation_df['recomendada']= recommendation_df['weighted_average_recommendation_score'] >= 4


In [23]:

#recommendation_df['movieId'] = recommendation_df.index
recommendation_df

,weighted_average_recommendation_score,movieId,recomendada
movieId,,,
hs1384,5.0,hs1384,True
ns6485,5.0,ns6485,True
ns6584,5.0,ns6584,True
as1906,5.0,as1906,True
ns6576,5.0,ns6576,True
...,...,...,...
hs545,0.5,hs545,False
as2309,0.5,as2309,False
as4551,0.5,as4551,False


In [24]:
dfr = recommendation_df.reset_index(drop=True)
#dfr = recommendation_df.set_index('weighted_average_recommendation_score')


In [25]:
dfr

,weighted_average_recommendation_score,movieId,recomendada
0,5.0,hs1384,True
1,5.0,ns6485,True
2,5.0,ns6584,True
3,5.0,as1906,True
4,5.0,ns6576,True
...,...,...,...
19230,0.5,hs545,False
19231,0.5,as2309,False
19232,0.5,as4551,False
19233,0.5,ns4164,False


In [26]:
dfr.to_csv('user_pearson.csv')

In [33]:
#dfr_limpio= dfr['recomendada'].drop(labels=None, axis=0)

dfr_limpio= dfr.where(dfr.recomendada == True)
dfr_limpio.dropna(inplace=True)


In [34]:
dfr_limpio

,weighted_average_recommendation_score,movieId,recomendada
0,5.0,hs1384,True
1,5.0,ns6485,True
2,5.0,ns6584,True
3,5.0,as1906,True
4,5.0,ns6576,True
...,...,...,...
5952,4.0,as6281,True
5953,4.0,as4068,True
5954,4.0,as1400,True
5955,4.0,as4640,True


In [35]:
def recomendacion (pelicula):
    peliculatitulo= movies_df[movies_df['title']==pelicula]
    idpelicula = peliculatitulo['id']
    #comparacionpel = dfr[dfr['movieId']].where(idpelicula == dfr['movieId'])
    
    
    if str(idpelicula) in dfr_limpio['movieId']:
        return  'Esta peli e spara vos!'
    else:
        return "segui buscando mejores opciones"

In [36]:
dfr_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5957 entries, 0 to 5956
Data columns (total 3 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   weighted_average_recommendation_score  5957 non-null   float64
 1   movieId                                5957 non-null   object 
 2   recomendada                            5957 non-null   object 
dtypes: float64(1), object(2)
memory usage: 186.2+ KB


In [37]:
recomendacion('the grand seduction')

'segui buscando mejores opciones'

Necesitamos aislar las peliculas que tengan rating 5 que excluya las peliculas que ya ha visto el usuario y en caso de quer la pelicula ingresada se encuentre en este dataframe devuelva un string que diga se recomienda 